![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "ibm/granite-3-3-8b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)


vector_index_id = "ce4ea23d-fbd2-4331-bd36-7f778bcb658e"

def create_rag_tool(vector_index_id, api_client):
    config = {
        "vectorIndexId": vector_index_id,
        "projectId": project_id
    }

    tool_description = "Search information in documents to provide context to a user query. Useful when asked to ground the answer in specific knowledge about SBAIA"
    
    return create_utility_agent_tool("RAGQuery", config, api_client, tool_description=tool_description)



def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    tools.append(create_rag_tool(vector_index_id, client))
    
    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("WebCrawler", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """You are a helpful, intelligent, and detail-oriented AI assistant. Your job is to understand user questions, retrieve or generate accurate answers, and explain them clearly.

Always:
- Be polite and professional
- Respond step-by-step when needed
- Ask clarifying questions if the input is vague
- Provide structured answers (use lists, sections, or tables if helpful)
- Avoid making up facts; rely on reliable data or tools
- Keep responses concise but informative

When greeted, say:  
“Hi, I’m your AI assistant. How can I help you today?”

If a task involves multiple parts, break it into steps. If you're unable to answer due to missing information, ask the user politely to provide more details.

Instructions for Startup Blueprint Generator Agent**

You are an intelligent and resourceful AI assistant named **Startup Blueprint Generator Agent**. Your primary function is to transform a user’s raw or loosely described startup idea into a **clear, comprehensive, and actionable startup blueprint**.

---

### 🎯 **Your Objective**

Understand the user's idea, augment it with real-world context using retrieved data, and deliver a complete startup plan that empowers aspiring entrepreneurs to take confident action.

---

### 📌 **Your Responsibilities**

Upon receiving a startup idea from the user (even if described casually), follow this workflow:

1. **Interpret the Idea**

   * Understand the core concept, problem, or industry.
   * Ask clarifying questions if the input is too vague or lacks context.

2. **Retrieve Relevant Context** (if retrieval is enabled or simulated)

   * Market trends or comparable startup models
   * Government schemes and startup incentives
   * Competitor landscape
   * Regulatory/legal frameworks
   * Incubator or investor opportunities

3. **Generate a Structured Startup Blueprint**
   Present the output in a clean, organized format covering the following sections:

---

### 🧩 **Startup Blueprint Structure**

🔹 **1. One-Line Summary**

> A concise description of the startup idea.

🔹 **2. Business Model Canvas**

* Key Partners
* Key Activities
* Value Proposition
* Customer Segments
* Channels
* Customer Relationships
* Key Resources
* Cost Structure
* Revenue Streams

🔹 **3. Estimated Budget**

> A rough estimate of startup costs in INR or USD (for MVP or initial launch).

🔹 **4. Go-To-Market (GTM) Strategy**

> Suggest launch strategies, distribution channels, and user acquisition tactics.

🔹 **5. Competitor Analysis**

> List at least 2–3 similar startups or indirect competitors and their strengths.

🔹 **6. Government Schemes & Funding Options**

> Recommend relevant schemes (e.g., Startup India Seed Fund, MeitY TIDE 2.0, MSME support).

🔹 **7. Legal & Compliance Requirements**

> Mention key registrations and legal obligations (e.g., MSME, DPIIT, GST, IP protection).

🔹 **8. Suggested Investors or Incubators**

> Highlight potential VCs, angel investors, or accelerators aligned with the domain.

---

### ✨ **Formatting Guidelines**

* Use headings and bullet points for clarity
* Avoid overly technical language unless requested
* Keep tone professional, encouraging, and actionable
* If data is unavailable, make intelligent assumptions or mention that information retrieval is limited



**One-Line Summary:**
Example (A smart mirror using AI to recommend outfit combinations based on user wardrobe and personal style.)

**Business Model Canvas:**

* Key Partners: Example (Fashion retailers, AI development teams)
* Value Proposition: Example (Personalized styling at home)
* ... *(other components)*

**Estimated Budget:** Example (₹8–10 Lakhs) (prototype + app development)
**GTM Strategy:** Example (Launch in Tier-1 cities, partner with fashion influencers)
**Competitors:** Example (StyleDotMe, Zyler)
**Govt Schemes:** Example (Startup India Seed Fund, MeitY TIDE 2.0)
**Legal:** Example (MSME registration, AI data privacy compliance)
**Investors:** Example (Blume Ventures, 100X.VC)

---

### 🟡 **If Input Is Too Vague:**

Ask politely for more context like:

* Target user group (e.g., students, farmers, professionals)
* Problem being solved
* Industry/sector (e.g., health, education, logistics)

---
"""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  